## One Hot Encoding of document "Item"s

In [1]:
import glob
import cPickle as pk
import pandas as pd
import numpy as np

In [21]:
def get_itemset(table):
    all_items = table['items'].apply( eval ).values
    if (all_items is not None) and (len(all_items) > 0):
        return set(reduce(lambda x, y: x + y, all_items))
    else:
        return []

class ItemMap(object):
    
    NAME_TEMPL = 'item_{i}'
    
    def __init__(self):
        self.ITEM_MAP = {}
    
    def __getitem__(self, item):
        if item not in self.ITEM_MAP:
            self.ITEM_MAP[item] = self.NAME_TEMPL.format(len(self.ITEM_MAP))
        
        return self.ITEM_MAP[item]

In [5]:
csv_files = glob.glob('../data/parsed/*.csv')

len(csv_files)

1500

In [22]:
# Get list of all items
itemset = set()

for _filepath in csv_files:
    with open(_filepath, 'rb') as f:
        _data = pd.read_csv(f)

    itemset.update( get_itemset(_data) )


itemset2 = filter(None, itemset)

len(itemset2)

Manually correct `itemset2` and lowercase items

In [2]:
itemset_2 = {'Completion of Acquisition or Disposition of Assets',
             'Regulation FD Disclosure',
             'Change of Servicer or Trustee',
             'Temporary Suspension of Trading Under Registrant\'s Employee Benefit Plans',
             'Triggering Events That Accelerate or Increase a Direct Financial Obligation or an Obligation under an Off-Balance Sheet Arrangement',
             'Financial statements and exhibits',
             'Failure to Make a Required Distribution',
             'Bankruptcy or Receivership',
             'Other Events',
             'Resignations of registrant\'s directors',
             "Amendments to the Registrant's Code of Ethics or Waiver of a Provision of the Code of Ethics",
             'Acquisition or disposition of assets',
             'Termination of a Material Definitive Agreement',
             'Triggering Events That Accelerate or Increase a Direct Financial Obligation under an Off-Balance Sheet Arrangement',
             'Mine Safety - Reporting of Shutdowns and Patterns of Violations',
             'Change in Credit Enhancement or Other External Support',
             'Unregistered Sales of Equity Securities',
             'Change in fiscal year',
             'Change in Shell Company Status',
             'Results of Operations and Financial Condition',
             'Material Modifications to Rights of Security Holders',
             'Changes in Control of Registrant',
             'Changes in control of registrant',
             'Departure of Directors or Principal Officers; Election of Directors; Appointment of Principal Officers',
             'Amendments to Articles of Incorporation or Bylaws; Change in Fiscal Year',
             'Creation of a Direct Financial Obligation or an Obligation under an Off-Balance Sheet Arrangement of a Registrant',
             'Shareholder Nominations Pursuant to Exchange Act Rule 14a-11',
             'Non-Reliance on Previously Issued Financial Statements or a Related Audit Report or Completed Interim Review',
             'Entry into a Material Definitive Agreement',
             'Other events',
             "Changes in Registrant's Certifying Accountant",
             'Financial Statements and Exhibits',
             'Submission of Matters to a Vote of Security Holders',
             "Changes in registrant's certifying accountant",
             'Securities Act Updating Disclosure',
             'Bankruptcy or receivership',
             'Departure of Directors or Certain Officers; Election of Directors; Appointment of Certain Officers: Compensatory Arrangements of Certain Officers',
             'Cost Associated with Exit or Disposal Activities',
             'Material Impairments',
             'Changes in Registrant.s Certifying Accountant',
             'Amendments to the Registrant.s Code of Ethics or Waiver of a Provision of the Code of Ethics',
            }

Save into a pickle

In [46]:
itemset_final = { it.lower() for it in itemset_2 }

with open('../data/OHE_items/all_items.pk', 'wb') as f:
    pk.dump(itemset_final, f)

In [3]:
with open('../data/OHE_items/all_items.pk', 'rb') as f:
    itemset = pk.load(f)

itemlist = list(itemset)

In [4]:
len(itemset)

36

In [23]:
%%time
all_in_one = pd.DataFrame()

for _filepath in csv_files:
    with open(_filepath, 'rb') as f:
        _data = pd.read_csv(f)[['timestamp', 'ticker', 'items']]

    all_in_one = pd.concat([all_in_one, _data], ignore_index=True)

CPU times: user 1min 24s, sys: 11.2 s, total: 1min 35s
Wall time: 1min 39s


In [24]:
all_in_one.shape

(195140, 3)

In [69]:
%%time

ITEMS = {}
for i,it in enumerate(itemlist):
    ITEMS['item_{}'.format(i)] = it
    all_in_one['item_{}'.format(i)] = 1.*all_in_one['items'].apply(lambda x: it in map(lambda x: x.lower(), eval(x)))

CPU times: user 1min 7s, sys: 932 ms, total: 1min 8s
Wall time: 1min 8s


In [71]:
all_in_one.head(3)

,timestamp,ticker,items,item_0,item_1,item_2,item_3,item_4,item_5,item_6,...,item_26,item_27,item_28,item_29,item_30,item_31,item_32,item_33,item_34,item_35
0,2002-05-17 17:27:05,A,"['Other events', 'Financial statements and exh...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2002-08-06 15:45:23,A,"['Other events', 'Financial statements and exh...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2002-08-19 21:47:23,A,"['Other events', 'Financial statements and exh...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
all_in_one.shape

(195140, 39)

In [72]:
all_in_one.to_csv('../data/OHE_items/item_OHE.csv')

In [5]:
# Save the ITEM_MAP as well
itemlist = list(itemset)
ITEMS = {}
for i,it in enumerate(itemlist):
    ITEMS['item_{}'.format(i)] = it


In [7]:
with open('../data/OHE_items/ITEM_MAP.pk', 'wb') as f:
    pk.dump(ITEMS, f)